# Description

Use this notebook to test the locally deployed trained LSTM model.

To run
1. Set required arguments in this notebook
    
    - `n_lags` is the timestep used in the trained model
    - `port` is the port set in `app.py`
    
2. Launch the Flask application by going to your terminal, `cd` into the appropriate working directory, and run the command `python3 app.py`

    - This starts the service at "`http://127.0.0.1:{port}`"
    
3. Prepare the input data

    - The structure depends on how it was defined in `app.py`
    - As of now, needs to be a list of lists where each sub list is of length `n_lags`
    - We must supply a list because we set up the API to expect (via a `POST`) a JSON object (and neither numpy arrays nor tensors are JSON serializable)
    
4. Send a POST request to the deployed model and retrieve the response (i.e., the predictions).

In [ ]:
import json

import torch
import requests

## 0. Preparation

In [ ]:
# Set required variables
n_lags = 7 # from trained model, change if different from 7
port = 1234 # change if different in app.py

In [ ]:
# Generate some test data to submit to deployed model
in_data = torch.tensor([[x for x in range(n_lags)],
                  [x for x in range(n_lags)]])
print(f"in_data is of type {type(in_data)} and of size {in_data.size()}")

## 1. Prepare data for POST submission

In [ ]:
# Input data cannot be a tensor so convert to list
in_data = in_data.tolist()
print(f"in_data is of type {type(in_data)}")

In [ ]:
# Deployment expects a JSON object with a single key "input_data" and value (the data/list)
body = {'input_data':in_data}

# Use this to make sure the input is JSON serializable (i.e., a valid JSON object)
try:
    _ = json.dumps(body)
    print("Input is a valid JSON object")
except Exception as e:
    print(e)

## 2. Submit POST request to generate predictions

- Don't forget, your application *must* already be running for this to work

In [ ]:
# The endpoint we set up in app.py is at /predict
url = f"http://localhost:{port}/predict"
resp = requests.post(url=url,
                     json=body) # using json arg sets POST up to submit a JSON object
if resp.status_code == 200:
    print('Success...API returned a response from the deployed model!')
else:
    print(f'Response returned a {resp.status_code} error code...with reason {resp.reason}')

## 3. View response/predictions

In [ ]:
preds = resp.json() # to quickly convert response to JSON
preds

In [ ]:
preds_tensors = torch.tensor(preds['output'])
preds_tensors

## 4. A quick experiment

1. Save the **_unscaled_** training data you used when training the model. You can use the following code in the training notebook.

```
import pickle

# x_train, y_train are the training data
training_data = {'x_train':x_train, 'y_train':y_train}
with open('train_data_lag7.pkl', 'wb') as f:
    pickle.dump(training_data, f)
```

2. Load the data in this notebook to submit to the deployed model. You can use this code to open the data.

```
import pickle

with open('train_data_lag7.pkl', 'rb') as f:
    training_data = pickle.load(f)
    
x_train = training_data['x_train']
y_train = training_data['y_train']
```

3. Use `x_train` in the input payload. Careful here...depending at which point you saved the data, you could have a 2D or 3D tensor. At the moment, the deployment expects a 2D input so if you have a 3D tensor, you have to remove that third dimension (e.g., with `squeeze()`) before you convert the tensor to a list.

4. Hopefully you find that the generated predictions returned from the deployed model are close to `y_train`.